In [ ]:
import os
from openmm import Platform, LangevinMiddleIntegrator, XmlSerializer
import openmm.app as app
from openmm.unit import picosecond, kelvin
import parmed
import MDAnalysis as md
from MDAnalysis.analysis import dihedrals
import nglview as ng
import numpy as np
import matplotlib.pyplot as plt
import sys

In [ ]:
data_path = 'data/'

In [ ]:
# Input and options
pdbfile = 'ala15.pdb'
psffile = 'ala15.psf'
pdb = app.PDBFile(os.path.join(data_path, pdbfile))
print(pdb.topology)

# 感觉用 amber14/protein.ff14SB.xml 似乎会更好
# 但实际上看过 amber14-all 就知道它包含了 amber14/protein.ff14SB.xml
forcefield = app.ForceField('amber14-all.xml')

nonbondedMethod = app.NoCutoff
constraints = None
rigidWater = False

dt = 0.002*picosecond
temperature = 300.15*kelvin
friction = 1/picosecond

steps = 50000
logInterval = 100
platform = Platform.getPlatformByName('OpenCL')
# platform = Platform.getPlatformByName('CPU')
# platformProperties = {'Precision': 'mixed'} # use this for CUDA
platformProperties = dict() # use this for CPU

In [ ]:
# Prepare the Simulation
print('Building system...')
topology = pdb.topology
print(topology.getPeriodicBoxVectors())
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod, constraints=constraints, rigidWater=rigidWater)
print(system.getDefaultPeriodicBoxVectors())
print(system.getForce(0))
print(system.getForce(1))
print(system.getForce(2))
print(dir(system.getForce(0)))
print(system.getForce(0).getBondParameters(0))

# modified by jk
# from openmm import unit
# edge = 1e20 * unit.nanometers
# system.setDefaultPeriodicBoxVectors([edge,0,0], [0,edge,0], [0,0,edge])
# print(system.getDefaultPeriodicBoxVectors())

# save the system
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
simulation = app.Simulation(topology, system, integrator, platform, platformProperties)
# save the topology and the contex
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(temperature, 2023)

# save psf file for future visualization
parmed_structure = parmed.openmm.load_topology(topology, system, positions)
parmed_structure.save(os.path.join(data_path, psffile), overwrite=True)

In [ ]:
with open(os.path.join(data_path, 'state_before_min.xml'), 'w') as f:
    f.write(
        XmlSerializer.serialize(
            simulation.context.getState(getPositions=True,
                                        getVelocities=True,
                                        getForces=True,
                                        getEnergy=True,
                                        getParameters=True,
                                        getParameterDerivatives=True,
                                        getIntegratorParameters=True,
                                        enforcePeriodicBox=True)))

In [ ]:
# Minimize
print("\nInitial system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())
print('Performing energy minimization...')
simulation.minimizeEnergy()
print("\nMinimized system energy")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

In [ ]:
# save file for future use
with open(os.path.join(data_path, 'system.xml'), 'w') as f:
    f.write(XmlSerializer.serialize(system))
with open(os.path.join(data_path, 'state.xml'), 'w') as f:
    f.write(XmlSerializer.serialize(
            simulation.context.getState(
            getPositions=True, getVelocities=True, getForces=True, 
            getEnergy=True, getParameters=True, getParameterDerivatives=True, 
            getIntegratorParameters=True, enforcePeriodicBox=True)))

In [ ]:
# topology from pdbfile, system and state from xml
# use the following code to load simulation to run

# integrator = LangevinMiddleIntegrator(temperature, friction, dt)
# system = XmlSerializer.deserialize(open('system.xml').read())
# simulation = app.Simulation(topology, system, integrator, platform, platformProperties)
# simulation.context.setState(XmlSerializer.deserialize(open('state.xml').read()))

In [ ]:
print('Simulating...')
simulation.currentStep = 0
dcdReporter = app.DCDReporter(os.path.join(data_path, 'traj-nocutoff.dcd'), logInterval)
dataReporter = app.StateDataReporter(os.path.join(data_path, 'traj-nocutoff.log'), logInterval, totalSteps=steps,
    step=True, time=True, speed=True, progress=True, elapsedTime=True, remainingTime=True, potentialEnergy=True, temperature=True, separator='\t')
stdoutReporter = app.StateDataReporter(sys.stdout, logInterval*10, totalSteps=steps,
    step=True, time=True, speed=True, progress=True, elapsedTime=True, remainingTime=True, potentialEnergy=True, temperature=True, separator='\t')
simulation.reporters.append(dataReporter)
simulation.reporters.append(stdoutReporter)
simulation.reporters.append(dcdReporter)
simulation.step(steps)

# Write file with final simulation state
simulation.saveState(os.path.join(data_path, 'final_state_nocutoff.xml'))

In [ ]:
u = md.Universe(os.path.join(data_path, psffile), os.path.join(data_path, 'traj-nocutoff.dcd'))
ng.show_mdanalysis(u, gui=True)

In [ ]:
N_terminus = u.select_atoms('resid 1 and name N')
C_terminus = u.select_atoms('resid 5 and name C')

## go through the whole trajectory and compute distance between them for every frame
dist = []
for frame in u.trajectory:
    dist.append(np.linalg.norm(N_terminus.positions - C_terminus.positions))

## the result is in the dist array    
dist = np.array(dist)
plt.figure(figsize=(15,5))

plt.plot( dist, '-k' )
plt.xlabel('timesteps')
plt.ylabel('end-to-end distance, A')

plt.show()


In [ ]:
ram1 = dihedrals.Ramachandran(u).run(0, 100) 
ram_mid = dihedrals.Ramachandran(u).run(200, 300)
ram2 = dihedrals.Ramachandran(u).run(400, 500) 

## ramachandran plot
fig, ax = plt.subplots(figsize=(8,8))
ram1.plot(ax=ax, color='r', marker='.', label='first 100 snapshots')
ram_mid.plot(ax=ax, color='g', marker='.', label='middle 100 snapshots')
ram2.plot(ax=ax, color='b', marker='.', label='last 100 snapshots')
ax.arrow(20, 20, -40, -40, width=2, head_width=8, head_length=12, fc='b', ec='b')
ax.text(30, 20, 'alpha region', color='blue', fontsize=20)
ax.arrow(20, 150, -40, 0, width=2, head_width=8, head_length=12, fc='k', ec='k')
ax.text(30, 150, 'beta region', fontsize=20)
plt.legend()
plt.show()